In [7]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [8]:
from sklearn.pipeline import make_pipeline

In [9]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/3', creation_time=1766468698523, experiment_id='3', last_update_time=1766468698523, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [10]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [11]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [56]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val)#, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, name="model")
    # mlflow.log_artifact(pipeline, name="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 45.64114429815588
🏃 View run languid-dolphin-326 at: http://127.0.0.1:5000/#/experiments/3/runs/5e57165fbc3040389679ec54b0b5232d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


In [57]:
from mlflow.tracking import MlflowClient


In [58]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '28c0fe9f3ff144359d7b1a9d544ab98c'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [59]:
client.get_experiment_by_name('green-taxi-duration')
# client.list_artifacts(RUN_ID)

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/3', creation_time=1766468698523, experiment_id='3', last_update_time=1766468698523, lifecycle_stage='active', name='green-taxi-duration', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [60]:
# path = client.download_artifacts(run_id=RUN_ID, path='model')


In [61]:
print(path)

/tmp/tmpvas_mqsr/model/


In [62]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

IsADirectoryError: [Errno 21] Is a directory: '/tmp/tmpvas_mqsr/model/'

In [ ]:
dv

NameError: name 'dv' is not defined